In [ ]:
# NEED TO INSTALL psycopg2 PACKAGE FOR POSTGRESQL CONNECTION
import json
import psycopg2
import psycopg2.extras
from psycopg2 import IntegrityError
from datetime import datetime
import os

conn_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "jhyu1982.1",
    "host": "prepopulation.c9k0s64g0471.eu-north-1.rds.amazonaws.com"
}

artist_file_path = 'unprocessed_artists_fromplaylistingest.txt'
song_file_path = 'unprocessed_songs_fromplaylistingest.json'

# Function to append a line to a text file
def append_to_file(file_path, line):
    with open(file_path, 'a') as file:
        file.write(line + '\n')

def append_song_to_file(file_path, song):
    try:
        with open(file_path, 'r+') as file:
            try:
                file_data = json.load(file)
            except json.JSONDecodeError:  # Handle empty file or invalid JSON
                file_data = []
            file_data.append(song)
            file.seek(0)
            file.truncate()  # Clear the file before writing updated content
            json.dump(file_data, file, indent=4)
    except FileNotFoundError:
        with open(file_path, 'w') as file:
            json.dump([song], file, indent=4)

# Function to insert data into the songTable
def insert_song_data(data):
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()  
    
    for item in data['items']:
        track = item['track']
        song = {
            "dateAddedToDB": str(datetime.now().date()),
            "dateLastModified": str(datetime.now().date()),
            "mainartistSpotifyID": track['artists'][0]['id'],
            "songSpotifyID": track['id'],
            "songTitle": track['name'],
            "songDuration": track['duration_ms'],
            "songAlbumType": track['album']['album_type'].upper(),
            "albumID": track['album']['id'],
            "explicit": track['explicit'],
            "songPopularity": track['popularity'],
            "previewURL": track['preview_url'] if track['preview_url'] is not None else "",
            "trackFeaturesAdded": False
        }
        
        insert_sql = """
        INSERT INTO songTable (dateAddedToDB, dateLastModified, mainartistSpotifyID, songSpotifyID, songTitle, songDuration, 
        songAlbumType, albumID, explicit, songPopularity, previewURL, trackFeaturesAdded) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (songSpotifyID) DO NOTHING;
        """
        
        try:
            cursor.execute(insert_sql, tuple(song.values()))
        except IntegrityError as e:
            connection.rollback()  # Rollback the transaction on error
            # Log or print e here to see the actual error message
            print(track['name'] + ": error adding to DB")
            print("IntegrityError:", e)
            # Assuming we decide to catch all IntegrityErrors for simplicity
            append_to_file(artist_file_path, song["mainartistSpotifyID"])
            append_song_to_file(song_file_path, song)
        except Exception as e:
            connection.rollback()  # Rollback for any other exception
            print("Unexpected exception:", e)
        else:
            connection.commit()  # Commit on success inside the try block

    cursor.close()
    connection.close()

# Function to read JSON file and process data
def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        insert_song_data(data)

def process_all_json_files(directory, base_filename):
    i = 1  # Start with the first file
    while True:
        file_path = os.path.join(directory, f'{base_filename}_{i}.json')
        if os.path.exists(file_path):
            print(f'Processing {file_path}...')
            try:
                process_json_file(file_path)
                print(f'Finished processing {file_path}')
            except Exception as e:
                print(f'Error processing {file_path}: {e}')
        else:
            print(f'No more files to process after {file_path}')
            break
        i += 1

In [ ]:
# change the directory to the directory where the json files are located
directory = 'C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\database_code\\songJsonTopSongs'

# change the base_filename to the base name of the json files, for example playlist_items_1.json, playlist_items_2.json, etc. will be playlist_items
base_filename = 'ARTISTBANK_items'

process_all_json_files(directory, base_filename)